In [1]:
import os
os.chdir("../")

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    load_model_path: Path
    load_test_data: Path
    local_file: Path


In [3]:
from PhishingDomainDetection.constants import CONFIG_FILE_PATH, PARAMS_FILE_APTH
from PhishingDomainDetection.utils import read_yaml_file, create_directories, save_reports
import joblib

In [4]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_APTH):
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_for_evaluation(self):
        self.data_load_config = self.config.training
        self.root_dir = self.data_load_config.root_dir
        self.file_dir = self.data_load_config.model_save
        self.file_path = os.path.join(self.root_dir, self.file_dir)
        return self.file_path

    def get_test_data_for_evaluation(self):
        self.data_load_config = self.config.training
        self.root_dir = self.data_load_config.root_dir
        self.file_dir = self.data_load_config.test_data_save
        self.file_path = os.path.join(self.root_dir, self.file_dir)        
        return self.file_path

    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        evaluation_config = EvaluationConfig(
            root_dir=Path(config.root_dir),
            load_model_path=Path(self.get_model_for_evaluation()),
            load_test_data=Path(self.get_test_data_for_evaluation()),
            local_file=Path(config.local_file)
        )
        
        return evaluation_config

In [5]:
aa = ConfigurationManager()
aa.get_evaluation_config()

[2023-02-13 16:27:24,410: INFO common]: yaml file configs\config.yaml load  successfully
[2023-02-13 16:27:24,420: INFO common]: yaml file params.yaml load  successfully
[2023-02-13 16:27:24,423: INFO common]: created directory at artifacts
[2023-02-13 16:27:24,426: INFO common]: created directory at artifacts/model_score


EvaluationConfig(root_dir=WindowsPath('artifacts/model_score'), load_model_path=WindowsPath('artifacts/training/model.pkl'), load_test_data=WindowsPath('artifacts/training/test_data.csv'), local_file=WindowsPath('score.json'))

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
import pandas as pd
import json


In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def load_test_data(self):
        datapath = self.config.load_test_data
        self.df = pd.read_csv(datapath)
    
    def load_model(self):
        model_path = self.config.load_model_path
        self.model = joblib.load(model_path)

    def evaluation(self):
        x_test = self.df.drop("phishing", axis=1)
        self.y_test = self.df['phishing']

        self.y_pred = self.model.predict(x_test)

    
    def save_score(self):
        test_accuracy = accuracy_score(self.y_test, self.y_pred)
        f1_model_score = f1_score(self.y_test, self.y_pred)
        scores = {
            'test_aaccuracy': test_accuracy,
            'f1_score': f1_model_score
        }
        
        evaluation_root_dir = self.config.root_dir
        score_file_dir = self.config.local_file
        raw_score_file_path = os.path.join(evaluation_root_dir, score_file_dir)

        save_reports(scores, raw_score_file_path)

In [10]:
config = ConfigurationManager()
evaluation_config = config.get_evaluation_config()
evaluation = Evaluation(config=evaluation_config)
evaluation.load_test_data()
evaluation.load_model()
evaluation.evaluation()
evaluation.save_score()

[2023-02-13 16:28:06,034: INFO common]: yaml file configs\config.yaml load  successfully
[2023-02-13 16:28:06,041: INFO common]: yaml file params.yaml load  successfully
[2023-02-13 16:28:06,043: INFO common]: created directory at artifacts
[2023-02-13 16:28:06,047: INFO common]: created directory at artifacts/model_score
report are saved in artifacts\model_score\score.json
